In [1]:
import cv2
import pytesseract
import re
import numpy as np
import pandas as pd
import joblib
import easyocr

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [58]:
def extract_img_2(path, keywords):
    img = cv2.imread(path)
    # Convert the image to gray scale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ocr_output = pytesseract.image_to_string(img_gray)
    dictionary = {}
    for line in ocr_output.split('\n'):
        if 'MCHC' in line :
            continue
            
        for keyword in keywords:
            try :
                try :
                    if re.search(keyword, line, re.IGNORECASE):
                        if '=' in line :
                            list_  = line.split('=')
                            list_ = str(list_)
                            result = re.search('\d+\.{0,1}\d*', list_).group()
                            dictionary[keyword] = float(result)
                            break
                        elif ':' in line :
                            list_  = line.split(':')
                            list_ = str(list_)
                            result = re.search('\d+\.{0,1}\d*', list_).group()
                            dictionary[keyword] = float(result)
                            break          
                except : 
                    if re.search(keyword, line, re.IGNORECASE):
                        list_  = str(line.split())
                        result = re.search('\d+\.{0,1}\d*', list_).group()
                        dictionary[keyword] = float(result)
                        break                    
            except :
                continue
                
    return dictionary

In [61]:
keywords = ['HB' , 'MCH' , 'MCV']
extracted_dic = extract_img_2('Sample\S1.jpg',keywords)
extracted_dic

{'HB': 13.6, 'MCV': 88.6, 'MCH': 29.5}

In [64]:
def extract_img_1(path, keywords):

        reader = easyocr.Reader(['en'],gpu=False) # this needs to run only once to load the model into memory
        img = path
        output = reader.readtext(img,detail=1)
        dictionary = {}
        for i in output:
            probability = i[-1]
            
            if probability >= 0.6:
                text = i[1] 
                
            if 'MCHC' in text :
                    continue

            for keyword in keywords :
                try :
                    try :
                        if re.search(keyword, text, re.IGNORECASE):
                            if '=' in text :
                                list_  = text.split('=')
                                list_ = str(list_)
                                result = re.search('\d+\.{0,1}\d*', list_).group()
                                dictionary[keyword] = float(result)
                                break
                            elif ':' in text :
                                list_  = text.split(':')
                                list_ = str(list_)
                                result = re.search('\d+\.{0,1}\d*', list_).group()
                                dictionary[keyword] = float(result)
                                break                            
                    except : 
                        if re.search(keyword, text, re.IGNORECASE):
                            list_  = str(text.split())
                            result = re.search('\d+\.{0,1}\d*', list_).group()
                            dictionary[keyword] = float(result)
                            break                    
                except :
                    continue
        return dictionary   

In [71]:
try : 
        try :

                    keywords = ['HB' , 'MCV' , 'MCH']
                    extracted_dic = extract_img_2('Sample\S1.jpg', keywords)
                    model = joblib.load('Model\GradientBoostingModel(anemia-m,f-3features)_2.h5')
                    features  = list(model.feature_names_in_)
                    data_dic = {}
                    data_dic[features[0]] = extracted_dic['HB']
                    data_dic[features[1]] = extracted_dic['MCH']
                    data_dic[features[2]] = extracted_dic['MCV']
                    data_dic[features[3]] = np.random.choice([0,1])
                    custom_data = pd.DataFrame(data = [data_dic] )   
                    print('-----'*10)
                    print('Your Input :\n',custom_data)
                    print('-----'*10)       
                    output = model.predict(custom_data)[0]       
                    print('\t Output :',output)

        except :  
                    keywords = ['HB' , 'MCV' , 'MCH']
                    extracted_dic = extract_img_1('Sample\S1.jpg',keywords)
                    model = joblib.load('Model\GradientBoostingModel(anemia-m,f-3features)_2.h5')
                    features  = list(model.feature_names_in_)
                    data_dic = {}
                    data_dic[features[0]] = extracted_dic['HB']
                    data_dic[features[1]] = extracted_dic['MCH']
                    data_dic[features[2]] = extracted_dic['MCV']
                    data_dic[features[3]] = np.random.choice([0,1])
                    custom_data = pd.DataFrame(data = [data_dic] )   
                    print('-----'*10)
                    print('Your Input :\n',custom_data)
                    print('-----'*10)       
                    output = model.predict(custom_data)[0]       
                    print('\t Output :',output)
except : 
        print('Your uploaded image can\'t be detected')
        print('-----'*10)       
        model = joblib.load('Model\GradientBoostingModel(anemia-m,f-3features)_2.h5')
        features  = list(model.feature_names_in_)        
        print('\t Enter Manually the Following Please : ',features[:-1])


--------------------------------------------------
Your Input :
    Hemoglobin   MCH   MCV  Gender_Male
0        13.6  29.5  88.6            1
--------------------------------------------------
Your uploaded image can't be detected
--------------------------------------------------
	 Enter Manually the Following Please :  ['Hemoglobin', 'MCH', 'MCV']


C:\Users\intel\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DummyClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\intel\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\intel\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or 

In [ ]:
class Anemia_image_pred :

    def __init__(self) :
      import cv2
      #import pytesseract
      import re
      import numpy as np
      import pandas as pd
      import joblib
      import easyocr

      #pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

    def extract_img_1(self , path, keywords):
        import easyocr
        import re      
        reader = easyocr.Reader(['en'],gpu=False) # this needs to run only once to load the model into memory
        img = path
        output = reader.readtext(img,detail=1)
        dictionary = {}
        for i in output:
            probability = i[-1]
            
            if probability >= 0.6:
                text = i[1] 
                
            if 'MCHC' in text :
                    continue

            for keyword in keywords :
                try :
                    try :
                        if re.search(keyword, text, re.IGNORECASE):
                            if '=' in text :
                                list_  = text.split('=')
                                list_ = str(list_)
                                result = re.search('\d+\.{0,1}\d*', list_).group()
                                dictionary[keyword] = float(result)
                                break
                            elif ':' in text :
                                list_  = text.split(':')
                                list_ = str(list_)
                                result = re.search('\d+\.{0,1}\d*', list_).group()
                                dictionary[keyword] = float(result)
                                break                            
                    except : 
                        if re.search(keyword, text, re.IGNORECASE):
                            list_  = str(text.split())
                            result = re.search('\d+\.{0,1}\d*', list_).group()
                            dictionary[keyword] = float(result)
                            break                    
                except :
                    continue
        return dictionary   


    def extract_img_2(self,path, keywords):
        import cv2
        import re
        import pytesseract
        pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

        img = cv2.imread(path)
        # Convert the image to gray scale
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        ocr_output = pytesseract.image_to_string(img_gray)
        dictionary = {}
        for line in ocr_output.split('\n'):
            if 'MCHC' in line :
                continue
                
            for keyword in keywords:
                try :
                    try :
                        if re.search(keyword, line, re.IGNORECASE):
                            if '=' in line :
                                list_  = line.split('=')
                                list_ = str(list_)
                                result = re.search('\d+\.{0,1}\d*', list_).group()
                                dictionary[keyword] = float(result)
                                break
                            elif ':' in line :
                                list_  = line.split(':')
                                list_ = str(list_)
                                result = re.search('\d+\.{0,1}\d*', list_).group()
                                dictionary[keyword] = float(result)
                                break          
                    except : 
                        if re.search(keyword, line, re.IGNORECASE):
                            list_  = str(line.split())
                            result = re.search('\d+\.{0,1}\d*', list_).group()
                            dictionary[keyword] = float(result)
                            break                    
                except :
                    continue
                    
        return dictionary


    def predict_image(self , path) :  
        import numpy as np
        import pandas as pd
        import joblib  
        import cv2
        import easyocr
        import re  
        import pytesseract
        pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"       
        
        try : 
                try :
                            keywords = ['HB' , 'MCV' , 'MCH']
                            extracted_dic = self.extract_img_2(path, keywords)
                            model = joblib.load('Model\GradientBoostingModel(anemia-m,f-3features)_2.h5')
                            features  = list(model.feature_names_in_)
                            data_dic = {}
                            data_dic[features[0]] = extracted_dic['HB']
                            data_dic[features[1]] = extracted_dic['MCH']
                            data_dic[features[2]] = extracted_dic['MCV']
                            data_dic[features[3]] = np.random.choice([0,1])
                            custom_data = pd.DataFrame(data = [data_dic] )   
                            #print('-----'*10)
                            #print('Your Input :\n',custom_data)
                            #print('-----'*10)       
                            output = model.predict(custom_data)[0]       
                            #print('\t Output :',output)
                            return output

                except :  
                            keywords = ['HB' , 'MCV' , 'MCH']                            
                            extracted_dic = self.extract_img_1(path,keywords)
                            model = joblib.load('Model\GradientBoostingModel(anemia-m,f-3features)_2.h5')
                            features  = list(model.feature_names_in_)
                            data_dic = {}
                            data_dic[features[0]] = extracted_dic['HB']
                            data_dic[features[1]] = extracted_dic['MCH']
                            data_dic[features[2]] = extracted_dic['MCV']
                            data_dic[features[3]] = np.random.choice([0,1])
                            custom_data = pd.DataFrame(data = [data_dic] )   
                            #print('-----'*10)
                            #print('Your Input :\n',custom_data)
                            #print('-----'*10)       
                            output = model.predict(custom_data)[0]       
                            #print('\t Output :',output)
                            return output
        except : 
                print('Your uploaded image can\'t be detected')
                print('-----'*10)       
                model = joblib.load('Model\GradientBoostingModel(anemia-m,f-3features)_2.h5')
                features  = list(model.feature_names_in_)        
                print('\t Enter Manually the Following Please : ',features[:-1])
          

In [ ]:
patiant = Anemia_image_pred()
patient_output = patiant.predict_image('Sample\S1.jpg')
correct_predication_name = {
                              'Anemia' : 'Normal Anemia' ,
                              'Good' : 'Normal ' , 
                              'Micro' : 'Microcytic Anemia' ,  
                              'Macro'	 : 'Macrocytic Anemia' ,
                              'CML' : 'Chronic Myelogenous Leukemia' ,
                              'Acute L' : 'Acute Lymphoblastic Leukemia' ,
                              'HyperTyroid' :  'Hyperthyroidism' , 
                              'Hypothyroid' : 'Hypothyroidism' ,
                              'Other Thyroid Abnormalities'  :  'Other Thyroid Abnormalities' ,
                              'Normal' :  'Normal ' , 	
                              'Hyperuricosuria (Gout)' : 'Hyperuricemia (Gout)' ,
                              'Hypouricosuria ' :  'Hypouricemia'  ,
                              'Jaundice' :  'Jaundice' , 
                              'Diabetic' : 'Diabetes' ,
                              'Pre Diabetic' : 'Prediabetes' ,
                              'Hypoglycemia' : 'Hypoglycemia' , 
                              'Prostatic_Cancer' :  'Prostatic Cancer' , 
                              'Rheumatiod_Arthities' : 'Rheumatoid Arthritis' , 
                              'Hypoparathyroid' : 'Hypoparathyroidism' ,
                              'Another Disease' :  'Another Disease' , 
                              'Hyperparathyroid' : 'Hyperparathyroidism' ,
                              'Acute  L or CML' :  'Acute Lymphoblastic Lekumia and Chronic Myelogenous Lekumia',
                              'Good' :  'Normal',
                              'Normal'  : 'Normal' 		}
                            
patient_output_disease = correct_predication_name[patient_output]
patient_output_disease                              